# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)

In [5]:
using LinearAlgebra
using LaTeXStrings
using DataStructures

In [6]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end


display_results (generic function with 1 method)

In [195]:

function Point_in_Neighborhood_infty(gamma, beta, A, b, x, lambda, s, mu, x_0, lambda_0, s_0, mu_0)
    rb = A*x - b
    rb0 = A*x_0 - b
    rc = A'*lambda + s - c
    rc0 = A'*lambda_0 + s_0 - c
    xs_pv = x .* s

    # tester conditions sur norm([rb, rc]) ou sur norm(rb) et norm(rc indépendemment) ?
     return sum(x.<=0)==false && sum(s.<=0)==false && sum(xs_pv.<=gamma*mu)==false &&
          norm(rb) <= norm(rb0)*beta*mu/mu_0 && norm(rc) <= norm(rc0)*beta*mu/mu_0
    #return sum(x.<=0)==false && sum(s.<=0)==false && sum(xs_pv.<=gamma*mu)==false &&
    #      norm(vcat(rb, rc)) <= norm(vcat(rb0, rc0))*beta*mu/mu_0 
end

function Next_Point_in_Neighborhood_infty(alpha_step, gamma, beta, A, b, x, lambda, 
                                        s, dir_k, mu, x_0, lambda_0, s_0, mu_0, n, m)
    x_next = x + alpha_step*dir_k[1:n]
    lambda_next = lambda + alpha_step*dir_k[n+1: n+m]
    s_next = s + alpha_step*dir_k[n+m+1: end]
    mu_next = s_next' * x_next / n
    return Point_in_Neighborhood_infty(gamma, beta, A, b, x_next, lambda_next, s_next, mu_next, x_0, lambda_0, s_0, mu_0)
end

function Next_Point_ArmijoCond(alpha, alpha_step, x, s, dir_k, mu, n, m)
    x_next = x + alpha_step*dir_k[1:n]
    s_next = s + alpha_step*dir_k[n+m+1: end]
    mu_next = s_next' * x_next / n
    return mu_next <= (1 - 0.001 * (alpha+alpha_step)) * mu
end

function UpdateAlphaOpt(gamma, beta, A, b, x, lambda, s, dir_k, mu, x_0, lambda_0, s_0, mu_0, alpha_step)
    alpha = 0
    x_k, lambda_k, s_k = x, lambda, s
    n = length(x)
    m = length(lambda)
    while Next_Point_in_Neighborhood_infty(alpha_step, gamma, beta,  A, b, x_k,lambda_k, 
            s_k, dir_k, mu, x_0, lambda_0, s_0, mu_0, n, m) == true && alpha+alpha_step <= 1 &&
            Next_Point_ArmijoCond(alpha, alpha_step, x_k, s_k, dir_k, mu, n, m) == true
            
        alpha += alpha_step
        x_k = x_k + alpha_step*dir_k[1:n]
        lambda_k = lambda_k + alpha_step*dir_k[n+1: n+m]
        s_k = s_k + alpha_step*dir_k[n+m+1: end]
    end
        
    println(alpha)
    return x_k, lambda_k, s_k        
end

UpdateAlphaOpt (generic function with 1 method)

In [223]:

function InfeasiblePathFollowing1(c, A, b, init_vect, gamma, beta, sigma_min, sigma_max, 
    max_iter, eps = 1e-8, alpha_step=1e-3)

    x_0, lambda_0, s_0 = init_vect
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    n_rows, n_cols = size(A) 
    
    rb_0 = A * x_0 - b
    rc_0 = A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    Xk = Diagonal(x_k)
    Sk = Diagonal(s_k)
    mu_k = s_k' * x_k / n_cols
    rb_k = A * x_k - b
    rc_k = A' * lambda_k + s_k - c
    
    k = 0
    pdd, cond_rb, cond_rc = 1, 1, 1
    e = ones(n_cols, 1)
    n_c = norm(c)
    n_b = norm(b)
    
    while k <= max_iter && pdd > eps && cond_rb > eps && cond_rc > eps  # 3 epsilon différents en pratique???

        sigma_k = sigma_min  # comment choisir sigma en pratique? ici je prends toujours sigma_min
        
        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [-rc_k
                -rb_k
                -Xk*Sk*e + sigma_k*mu_k*e]

        dir_k = Jacob_Fk\Fk

        x_k, lambda_k, s_k = UpdateAlphaOpt(gamma, beta, A, b, x_k, lambda_k, s_k, dir_k,
                                            mu_k, x_0, lambda_0, s_0, mu_0, alpha_step)
        
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = A' * lambda_k + s_k - c
        
        # valeurs pour les conditions d'arret:
        pdd = abs(c' * x_k - b' * lambda_k) / (1 + abs(c' * x_k))
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        
        k += 1
        
        # affichage
        println("---------------------------------------------------------------")
        println("Iter ", k, ":   pdd = ", pdd, " |  cond_rb = ", cond_rb, " |  cond_rc = ", cond_rc)
    end
    
    criteria = [k > max_iter,  pdd <= eps, cond_rb <= eps, cond_rc <= eps]
    criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps"]
    println("\n stopping criterion = ",criteria_names[findall(criteria)])
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

InfeasiblePathFollowing1 (generic function with 4 methods)

In [224]:
### probleme 1
c = [1, 0 ,0]
A = Matrix([1, 1, 1]')
b = [1];

In [225]:
# initialisation 1 
x01 = [10, 1, 3]
lambda01 = [-2]
s01 = [3, 4, 2]
init_vect1 = (x01, lambda01, s01);
gamma, beta = 0.3, 2  # pour ce problème, ne marche pas lorque gamma est trop grand.
res_ipf1 =  InfeasiblePathFollowing1(c, A, b, init_vect1, gamma, beta, 0.2, 0.3, 1000);
display_results(res_ipf1) 

0.9220000000000007
---------------------------------------------------------------
Iter 1:   pdd = 2.240667511777982 |  cond_rb = 0.5069999999999946 |  cond_rc = 0.07799999999989682
0.9990000000000008
---------------------------------------------------------------
Iter 2:   pdd = 1.6361624823453043 |  cond_rb = 0.0005070000000255703 |  cond_rc = 7.799999990998341e-5
0.9990000000000008
---------------------------------------------------------------
Iter 3:   pdd = 0.33493863063490076 |  cond_rb = 5.06999987215373e-7 |  cond_rc = 7.800000645374894e-8
0.9990000000000008
---------------------------------------------------------------
Iter 4:   pdd = 0.07221443409713485 |  cond_rb = 5.070025510534037e-10 |  cond_rc = 7.79987717886108e-11

 stopping criterion = ["cond_rb <= eps", "cond_rc <= eps"]

-----------------------------------------------------------------------
------------------------------- RESULTS -------------------------------


OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [0.0407371, 0.51803, 0.441233]
  "lambda_opt" => [-0.0344191]
  "s_opt"      => [1.03442, 0.0344191, 0.0344191]
  "n_iter"     => 4
  "pdd"        => 0.0722144
  "cond_rb"    => 5.07003e-10
  "cond_rc"    => 7.79988e-11

In [ ]:
# cond_rb et cond_rc trouvées dans le livre mais elles semblent arrêter très rapidement l'algorithme...